In [1]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
import pprint


from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor
from architectures import vgg16net, smallnet1, smallnet2
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, train_model, train_log, log_test_score,set_lossfn, train, hp_sweep

from architectures import vgg16net


In [2]:
wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [3]:
""""
grid search
'learning_rate': {
            'values': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]
"""

'"\ngrid search\n\'learning_rate\': {\n            \'values\': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]\n'

In [11]:
config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        'dropout':{
            'values': [0.5, 0.4, 0.3]
        },
        'weight_decay':{
            'values': [1e-5,2e-5, 3e-5,4e-5]
        },
        'epochs':{
            'value': 150
        },
        'lin_layer_size': {
            'values': [100] #, 150, 50
        },
        'first_lin_lay':{
            'values':[21504] #ed (1x21504 and 5120x100)
        },
        'optimizer': {
            'values': ['adam']
        },
        'scheduler': {
            'values': [0.1, 0.01, 0.001]
        },
        'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 5e-5,
                'max': 1e-4
        },
        'loss_fn': {
            'values': ['MSE', 'CrossEntropy']
        },
         'data_set':{
            'values':['Augmented']
        },
        'channels': {
            'values': [3]
        },
        'ks':{
            'values':[(3,5),(3,3)]
        },
            'num_classes': {
            'values': [11]
        },
         'model_name' : {
             'values': ['smallnet2']
        },
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']
        },
        }
    }
    

col_dict = {
    'colour': 'colour',
    'size': [452,144],
    'padding': 5
}

title = f'IDSW_7c3l_150_122023'
save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location': r'pickles/'}
                #r'/its/home/nn268/antvis/optics/

sweep_id = wandb.sweep(config, project=f"modelSize_7conv3lin_IDSW_UNwrapped_{col_dict['colour']}_{col_dict['size']}_pad_{col_dict['padding']}")

Create sweep with ID: 6gacrlkj
Sweep URL: https://wandb.ai/antvis/modelSize_7conv3lin_IDSW_UNwrapped_colour_%5B452%2C%20144%5D_pad_5/sweeps/6gacrlkj


In [ ]:
"""
config = dict(
    epochs= 80, 
    learning_rate =3.0779e-5,
    dataset= 'IDSW',
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    dropout = 0.4,
    first_lin_lay =4096,
    lin_layer_size= 100,
    ks = 3,
    in_chan = 3
)



col_dict = {
    'colour': 'colour',
    'size': [32,32]
}"""

In [ ]:
#pprint.pprint(config)

In [12]:
#sweep_id = wandb.sweep(config, project='HPS_basedon_decent-sweep-24_UNwrapped_3232')

device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda:1 device


ModuleNotFoundError: No module named 'fns4wadb'

In [ ]:
"""
                            # Common functions


def set_lossfn(lf):
    if lf =='MSE':
        loss_fn = nn.MSELoss()
    elif lf == 'CrossEntropy':
        loss_fn = nn.CrossEntropyLoss()
    return loss_fn

                                # HP Sweep

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        x_train, y_train, x_val, y_val, x_test, y_test = get_data()

        model = vgg16net(in_chan=3, f_lin_lay=5120, l_lin_lay=11).to(device) 
        if config.loss_fn == 'MSE':
            loss_fn = nn.MSELoss()
        elif config.loss_fn == 'CrossEntropy':
            loss_fn = nn.CrossEntropyLoss()
        
        e_count = 0
        #optimizer = build_optimizer(network, config.optimizer, config.learning_rate, config.weight_decay)
        #if e_count >= 20:
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)
        #else:
        #    optimizer = build_optimizer(model, config.optimizer, config.learning_rate)
        
        for epoch in range(config.epochs):

            t_loss, predict_list, t_num_correct, model, optimizer = loop(model, x_train, y_train, epoch, loss_fn, device, col_dict,num_classes=11, optimizer=optimizer)
            t_accuracy = (t_num_correct /len(x_train))*100
            v_loss, __, v_num_correct= loop(model, x_val, y_val, epoch, loss_fn, device,col_dict,num_classes=11, train=False) 
            v_accuracy= (v_num_correct / len(x_val))*100
            
            t_avg_loss =t_loss/len(x_train)
            v_avg_loss = v_loss /len(x_val)
            
            e_count +=1
            # logging
            wandb.log({'avg_train_loss': t_avg_loss, 'epoch':epoch})
            wandb.log({'avg_val_loss': v_avg_loss, 'epoch':epoch})
            wandb.log({'train_loss': t_loss, 'epoch':epoch})
            wandb.log({'val_loss': v_loss, 'epoch':epoch})
            wandb.log({'train_accuracy_%': t_accuracy, 'epoch':epoch})
            wandb.log({'val_accuracy_%': v_accuracy, 'epoch':epoch})"""
            

"""

def pipeline(hp, col_dict, device): 
    
    title = f"{col_dict['colour']}_Wrapped_3232"
    
    x_train, y_train, x_val, y_val, x_test, y_test = get_data()
    
    with wandb.init(project=title, config=hp):
        config = wandb.config
        model = build_net(lin_layer_size =config.lin_layer_size,
                          dropout=config.dropout,
                          first_lin_lay=config.first_lin_lay,
                          ks = config.ks,
                          in_chan = config.in_chan,
                          pad_size=2
                         ).to(device)
        #model = vgg16TorchNet().to(device)
        loss_fn = nn.MSELoss()
        
        train_model(model, x_train, y_train, x_val, y_val,loss_fn, config, col_dict, device, num_classes=11)
        test_loop(model, x_test, y_test, loss_fn, device, col_dict, title, num_classes=11)
        
    return model
"""

In [14]:
#model= pipeline(config, col_dict, device)
def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model = hp_sweep(config, col_dict, save_dict, device)

wandb.agent(sweep_id, tr, count=25)

wandb: Agent Starting Run: 20unsb6z with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	dropout: 0.5
wandb: 	epochs: 150
wandb: 	first_lin_lay: 21504
wandb: 	image_path: /its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 5]
wandb: 	learning_rate: 7.277472400391642e-05
wandb: 	lin_layer_size: 100
wandb: 	loss_fn: CrossEntropy
wandb: 	model_name: smallnet2
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.1
wandb: 	weight_decay: 2e-05


Traceback (most recent call last):
  File "/tmp/ipykernel_109219/861307837.py", line 6, in tr
    model = hp_sweep(config, col_dict, save_dict, device)
NameError: name 'hp_sweep' is not defined


Run 20unsb6z errored: NameError("name 'hp_sweep' is not defined")
wandb: ERROR Run 20unsb6z errored: NameError("name 'hp_sweep' is not defined")
wandb: Agent Starting Run: yfgzo888 with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	dropout: 0.4
wandb: 	epochs: 150
wandb: 	first_lin_lay: 21504
wandb: 	image_path: /its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 5]
wandb: 	learning_rate: 5.229931282278883e-05
wandb: 	lin_layer_size: 100
wandb: 	loss_fn: CrossEntropy
wandb: 	model_name: smallnet2
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.1
wandb: 	weight_decay: 1e-05


Traceback (most recent call last):
  File "/tmp/ipykernel_109219/861307837.py", line 6, in tr
    model = hp_sweep(config, col_dict, save_dict, device)
NameError: name 'hp_sweep' is not defined


Run yfgzo888 errored: NameError("name 'hp_sweep' is not defined")
wandb: ERROR Run yfgzo888 errored: NameError("name 'hp_sweep' is not defined")
wandb: Agent Starting Run: w58hhlt1 with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	dropout: 0.3
wandb: 	epochs: 150
wandb: 	first_lin_lay: 21504
wandb: 	image_path: /its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 3]
wandb: 	learning_rate: 8.424822216778041e-05
wandb: 	lin_layer_size: 100
wandb: 	loss_fn: MSE
wandb: 	model_name: smallnet2
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.1
wandb: 	weight_decay: 4e-05


Traceback (most recent call last):
  File "/tmp/ipykernel_109219/861307837.py", line 6, in tr
    model = hp_sweep(config, col_dict, save_dict, device)
NameError: name 'hp_sweep' is not defined


Run w58hhlt1 errored: NameError("name 'hp_sweep' is not defined")
wandb: ERROR Run w58hhlt1 errored: NameError("name 'hp_sweep' is not defined")
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [ ]:
wandb.agent(sweep_id, train, count=25)